In [ ]:
import numpy
import pylab as plt
import json
import torch
import torchvision
import pycuda.autoinit
import pycuda.driver as cuda
import tensorrt as trt
TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
trt.init_libnvinfer_plugins(TRT_LOGGER, '')
import common
import onnxruntime
onnxruntime.disable_telemetry_events()
from dalle import TextTokenizer
from utils_sr import *
from PIL import Image
from tqdm import tqdm
from dalle import VQGanDetokenizer

detokenizer = VQGanDetokenizer().eval()
detokenizer.load_state_dict(torch.load('models/detoker.pt'))

class HostDeviceMem(object):
    def __init__(self, host_mem):
        self.host = host_mem
        self.device = cuda.mem_alloc(host_mem.nbytes)
    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)
    def __repr__(self):
        return self.__str__()

def to_numpy(tensor):
    if isinstance(tensor, numpy.ndarray):
        return tensor
    else:
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def get_engine_info(engine0):
    for b in engine0:
        print(b, trt.nptype(engine0.get_binding_dtype(b)), trt.volume(engine0.get_binding_shape(b)) * engine0.max_batch_size)

runtime = trt.Runtime(TRT_LOGGER)
stream = cuda.Stream()
with open("engines/decoder0v2.trt32", mode="rb") as f:
    engine0 = runtime.deserialize_cuda_engine(f.read())
    context0 = engine0.create_execution_context()
with open("engines/decoder1v2.trt", mode="rb") as f:
    engine1 = runtime.deserialize_cuda_engine(f.read())
    context1 = engine1.create_execution_context()
with open("engines/decoder2v2.trt", mode="rb") as f:
    engine2 = runtime.deserialize_cuda_engine(f.read())
    context2 = engine2.create_execution_context()
with open("engines/decoder3v2.trt", mode="rb") as f:
    engine3 = runtime.deserialize_cuda_engine(f.read())
    context3 = engine3.create_execution_context()
image_count = 1
tAM = HostDeviceMem(cuda.pagelocked_empty(128 * image_count, numpy.int32))
tES = HostDeviceMem(cuda.pagelocked_empty(262144 * image_count, numpy.float32))
tIT = HostDeviceMem(cuda.pagelocked_empty(1 * image_count, numpy.int32))
tTI = HostDeviceMem(cuda.pagelocked_empty(1, numpy.int32))
tDS = HostDeviceMem(cuda.pagelocked_empty(4096 * image_count, numpy.float32))
tOut = HostDeviceMem(cuda.pagelocked_empty(32832 * image_count, numpy.float32))
tAS0 = HostDeviceMem(cuda.pagelocked_empty(12582912 * image_count, numpy.float32))
tAS1 = HostDeviceMem(cuda.pagelocked_empty(12582912 * image_count, numpy.float32))
tAS2 = HostDeviceMem(cuda.pagelocked_empty(12582912 * image_count, numpy.float32))
tAS3 = HostDeviceMem(cuda.pagelocked_empty(12582912 * image_count, numpy.float32))

In [ ]:
TEXT = "night beautiful norway fjord with blue river"
TEMPERATURE = 1.0
TOPK = 2048
TOPP = 0.1
SFACTOR = 32
SEED = 0
WIDTH = 256

In [ ]:
import random
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
numpy.random.seed(42)

with open('models/vocab.json', 'r', encoding='utf8') as f:
    vocab = json.load(f)
with open('models/merges.txt', 'r', encoding='utf8') as f:
    merges = f.read().split("\n")[1:-1]
ort_session0 = onnxruntime.InferenceSession('./onnx/encoder0/encoder0.onnx', providers=['CPUExecutionProvider'])
ort_session1 = onnxruntime.InferenceSession('./onnx/encoder1/encoder1.onnx', providers=['CPUExecutionProvider'])
tokenizer = TextTokenizer(vocab, merges)
WINDOW = list(range(256))
tokens = tokenizer.tokenize(TEXT, is_verbose=False)[:64]
text_tokens = numpy.ones((2, 64), dtype=numpy.int32)
text_tokens[0, :2] = [tokens[0], tokens[-1]]
text_tokens[1, :len(tokens)] = tokens
text_tokens = torch.tensor(
    text_tokens, 
    dtype=torch.long, 
)
ort_inputs = {ort_session0.get_inputs()[0].name: to_numpy(text_tokens)}
ort_outs = ort_session0.run(None, ort_inputs)
ort_inputs = {ort_session1.get_inputs()[0].name: to_numpy(ort_outs[0]), ort_session1.get_inputs()[1].name: to_numpy(text_tokens)}
ort_outs = ort_session1.run(None, ort_inputs)
encoder_state = torch.from_numpy(ort_outs[0])
seed_add = 0
expanded_indices = [0] * image_count + [1] * image_count
text_tokens = text_tokens[expanded_indices]
encoder_state = encoder_state[expanded_indices]
attention_mask = text_tokens.not_equal(1).long()
attention_state = torch.zeros(size=(24, image_count * 4, 256, 2048))
image_tokens = torch.full((WIDTH + 1, image_count), 16415, dtype=torch.long)
if SEED > 0: torch.manual_seed(SEED + seed_add)
token_indices = torch.arange(256).repeat(WIDTH // 256)
settings = torch.tensor([TEMPERATURE, TOPK, SFACTOR])
numpy.copyto(tAM.host, to_numpy(attention_mask).ravel())
numpy.copyto(tES.host, to_numpy(encoder_state).ravel())
numpy.copyto(tAS0.host, to_numpy(attention_state[:6]).ravel())
numpy.copyto(tAS1.host, to_numpy(attention_state[6:12]).ravel())
numpy.copyto(tAS2.host, to_numpy(attention_state[12:18]).ravel())
numpy.copyto(tAS3.host, to_numpy(attention_state[18:]).ravel())
cuda.memcpy_htod_async(tAM.device, tAM.host, stream)
cuda.memcpy_htod_async(tES.device, tES.host, stream)
cuda.memcpy_htod_async(tAS0.device, tAS0.host, stream)
cuda.memcpy_htod_async(tAS1.device, tAS1.host, stream)
cuda.memcpy_htod_async(tAS2.device, tAS2.host, stream)
cuda.memcpy_htod_async(tAS3.device, tAS3.host, stream)
total_logits = torch.zeros((WIDTH, 2, 16384)).float()
for i in tqdm(range(WIDTH)):
    numpy.copyto(tIT.host, to_numpy(image_tokens[i]).ravel())
    numpy.copyto(tTI.host, to_numpy(token_indices[[i]]).ravel())
    cuda.memcpy_htod_async(tIT.device, tIT.host, stream)
    cuda.memcpy_htod_async(tTI.device, tTI.host, stream)
    queue = [tAM, tES, tAS0, tIT, tTI, tDS, tAS0]
    context0.execute_async_v2(bindings=[v.device for v in queue], stream_handle=stream.handle)
    stream.synchronize()
    queue = [tAM, tES, tDS, tAS1, tTI, tDS, tAS1]
    context1.execute_async_v2(bindings=[v.device for v in queue], stream_handle=stream.handle)
    stream.synchronize()
    queue = [tAM, tES, tDS, tAS2, tTI, tDS, tAS2]
    context2.execute_async_v2(bindings=[v.device for v in queue], stream_handle=stream.handle)
    stream.synchronize()
    queue = [tAM, tES, tDS, tAS3, tTI, tAS3, tOut]
    context3.execute_async_v2(bindings=[v.device for v in queue], stream_handle=stream.handle)
    cuda.memcpy_dtoh_async(tOut.host, tOut.device, stream)
    stream.synchronize()
    logits = torch.from_numpy(tOut.host).reshape(2, 1, 16416)
    logits = logits[:, -1, : 2 ** 14]
    total_logits[i] = logits
    temperature = settings[[0]]
    top_k = settings[[1]].to(torch.long)
    supercondition_factor = settings[[2]]
    logits = logits[:image_count] * (1 - supercondition_factor) + logits[image_count:] * supercondition_factor
    logits = logits[0]
    _, pindices = logits.topk(logits.shape[0] - TOPK, largest=False)
    probas = logits.softmax(-1)
    min_val = min(probas.max().item(), TOPP)
    logits[pindices] = 0
    logits[probas < min_val] = 0
    image_tokens[i + 1] = torch.multinomial(logits.softmax(-1), 1)[0]

In [ ]:
with torch.cuda.amp.autocast(dtype=torch.float32) and torch.no_grad():
    z = image_tokens[1:].T.reshape(-1, 256).clone()
    z.clamp_(0, detokenizer.vocab_count - 1)
    # z = z.repeat(1,1,2)
    z = z.view([-1, 1, 2 ** 4, 2 ** 4])
    z = z.flatten(1, 2).transpose(1, 0).flatten(1, 2)
    z = z.flatten().unsqueeze(1)
    z = detokenizer.embedding.forward(z)
    z = z.view((1, 16, -1, 2 ** 8))
    z = torch.cat([z[:,:,:16], z[:,:,16:]], dim=2)
    z = torch.cat([z[:,:,:16], z[:,:,16:].flip(2)], dim=2)
    # z = z.roll(8, 2)
    t1 = z.shape[1]; t2 = z.shape[2]
    z = z.permute(0, 3, 1, 2).contiguous()
    z = detokenizer.post_quant_conv.forward(z)
    z = detokenizer.decoder.conv_in.forward(z)
    z = detokenizer.decoder.mid.forward(z, t1, t2)
    for i in range(4, -1, -1):
        z = detokenizer.decoder.up[i].forward(z, t1, t2)
    z = detokenizer.decoder.norm_out.forward(z)
    z *= torch.sigmoid(z)
    z = detokenizer.decoder.conv_out.forward(z)
    z = z.permute(0, 2, 3, 1)
    images = (z.clip(0.0, 1.0) * 255).cpu().detach()

In [ ]:
img = Image.fromarray((images[0].numpy()).astype(numpy.uint8))
img.save('output.png')
img

In [ ]:
del engine0, engine1, engine2, context0, context1, context2, stream
stream = cuda.Stream()
with open(f"engines/esrgan1x1.trt", mode="rb") as f:
    engine3 = runtime.deserialize_cuda_engine(f.read())
    context3 = engine3.create_execution_context()
inputs, outputs, bindings = common.allocate_buffers(engine3)
lr_image = Image.open('output.png').convert('RGB')
lr_image = np.array(lr_image)
lr_image = pad_reflect(lr_image, 15)
patches, p_shape = split_image_into_overlapping_patches(lr_image, patch_size=192, padding_size=24)
patches = np.swapaxes(np.swapaxes(patches, 1, 2), 1, 3)
numpy.copyto(inputs[0].host, patches.ravel() / 255.)
res = common.do_inference(context3, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)[0]
np_sr_image = res.reshape(-1, 1920, 1920, 3)
padded_size_scaled = tuple(np.multiply(p_shape[0:2], 8)) + (3,)
scaled_image_shape = tuple(np.multiply(lr_image.shape[0:2], 8)) + (3,)
np_sr_image = stich_together(np_sr_image, padded_image_shape=padded_size_scaled, target_shape=scaled_image_shape, padding_size=24 * 8)
sr_img = (np_sr_image*255).astype(np.uint8)
sr_img = unpad_image(sr_img, 15*8)
sr_img = Image.fromarray(sr_img)
sr_img.save('sr_output.png')
sr_img

In [ ]:
del engine0, engine1, engine2, context0, context1, context2, stream
stream = cuda.Stream()
with open(f"engines/srgan1x1.trt", mode="rb") as f:
    engine3 = runtime.deserialize_cuda_engine(f.read())
    context3 = engine3.create_execution_context()
inputs, outputs, bindings = common.allocate_buffers(engine3)
lr_image = Image.open('output.png').convert('RGB')
lr_image = np.array(lr_image)
numpy.copyto(inputs[0].host, numpy.moveaxis(lr_image[None], -1, 1).ravel() / 255.)
res = common.do_inference(context3, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)[0]
# np_sr_image = numpy.swapaxes(res.reshape(3, 256*4, 256*4), 0, -1)
np_sr_image = numpy.moveaxis(res.reshape(3, 256*4, -1), 0, -1).clip(0, 1)
sr_img = (np_sr_image*255).astype(np.uint8)
sr_img = Image.fromarray(sr_img)
sr_img.save('sr_output.png')
sr_img